In [1]:
from tkinter import filedialog
import cv2
import easygui as eg
import numpy as np

In [2]:
def image_file_explore():
    # Prompt user to select an image file
    file_path = filedialog.askopenfilename(
        title="Select an Image File",
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )
    if not file_path:
        eg.msgbox("No File Selected.", "Error")
    return file_path

In [3]:
#Reizing image
def resize(img, height=640):
    img_X_scale = height / img.shape[0]
    new_img_width = int(img.shape[1] * img_X_scale)
    img_resized = cv2.resize(img, (int(new_img_width), int(height)))
    return img_resized

In [4]:
def BiLateralFitlerDenoise(img, diameter = 9, sigma_color=75, sigma_space=75 ):
    return cv2.bilateralFilter(img, diameter, sigma_color, sigma_space)

In [5]:
def is_grayscale(image):
    if len(image.shape) == 2:
        return True
    if image.shape[2] == 1:#no of colour channels 
        return True
    
    channels = cv2.split(image)
    return np.allclose(channels[0], channels[1]) and np.allclose(channels[0], channels[2]) # checks if the values are close 

In [6]:
def unsharpMask(img, kernel_size=5, sigma=1.5, amount =1.0, threshold=0):
    blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), sigma, sigma)

    sharpened = cv2.addWeighted(img, 1 + amount, blurred, -amount, 0)

    # Apply a threshold to the sharpening
    image_sharp = np.where((img - blurred) < -threshold, 0, sharpened)

    return image_sharp

In [7]:
def enhanceContrast(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    if (is_grayscale(img)):#check if gray
        
        enhanced = clahe.apply(img)
        return enhanced
    else:
        # Convert to LAB color space
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        
        # Apply CLAHE to L channel
        enhanced_l = clahe.apply(l)
        
        # Merge and convert back
        enhanced_lab = cv2.merge([enhanced_l, a, b])
        return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

In [8]:
def video_file_explore():
    # Prompt user to select a video file
    file_path = filedialog.askopenfilename(
        title="Select a Video File",
        filetypes=[("Video files", "*.mp4 *.avi *.mov")]
    )
    if not file_path:
        eg.msgbox("No File Selected.", "Error")
        return None
    return file_path

In [9]:
def capture_image():
    image = cv2.VideoCapture(0)
    (check, image) = image.read()
    return image

In [10]:
def detect_faces_with_skin_mask(frame):
    # Convert to YCrCb color space to detect skin color
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    # Define skin color range in YCrCb
    lower = np.array([0, 133, 77], dtype=np.uint8)
    upper = np.array([255, 173, 127], dtype=np.uint8)
    # Create a mask for skin-colored regions
    skin_mask = cv2.inRange(ycrcb, lower, upper)
    return skin_mask

In [11]:
def detect_faces_with_skin_region(frame):
    # Generate a mask for the skin-colored region
    skin_mask = detect_faces_with_skin_mask(frame)
    # Extract the skin region from the original image
    skin_region = cv2.bitwise_and(frame, frame, mask=skin_mask)
    return skin_region

In [12]:
def get_skin_colour(image):
    skin= detect_faces_with_skin_region(image)
    skin_mask = detect_faces_with_skin_mask(image)
    
    # Calculate the avg color 
    skin_pxls = image[skin_mask > 0] # Get only skin pixels
    if len(skin_pxls) > 0:
        avg_colour = np.mean(skin_pxls, axis = 0)
    else:
        avg_colour = [0,0,0]  #Default if no skin is detected
    return avg_colour

In [13]:
def draw_skin_colour(image, skin_colour):
    colour = (int(skin_colour[0]), int(skin_colour[1]), int(skin_colour[2]))
    cv2.putText(image, "Skin Colour: ", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.rectangle(image, (150, 10), (180, 40), colour, -1)

In [14]:
def increase_brightness(colour, factor = 1.675):
    return tuple(min(int(c * factor), 255) for c in colour)

In [25]:
def skin_colour(image):
    avg_colour = get_skin_colour(image)
    img = increase_brightness(avg_colour)
    img = draw_skin_colour(image, img)
    return image

In [20]:
def capture_skin_based_roi(frame):
    # Generate the skin mask
    skin_mask = detect_faces_with_skin_mask(frame)

    # Find contours in the skin mask
    contours, _ = cv2.findContours(
        skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # If contours are found, proceed to find the largest skin region
    if contours:
        # Find the largest contour by area, which is likely the main skin region
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Define the ROI based on the bounding box of the largest contour
        roi = frame[y:y + h, x:x + w]

        return roi, (x, y, w, h)  # Return the ROI and bounding box coordinates
    else:
        print("No significant skin-colored region detected.")
        return None, None  # Return None if no skin region is found


In [21]:
def preprocessing(image):
    img = resize(image)
    #img = BiLateralFitlerDenoise(img)
    #img = enhanceContrast(img)
    img = unsharpMask(img)
    img = unsharpMask(img)
    return img

In [26]:

def main():
    while True:
        # Prompt user to choose between image or video upload or exit
        choices = ["1. Image Upload", "2. Video Upload", "3. Take Photo", "4. Show Colour", "5. Exit"]
        user_choice = eg.choicebox("Choose an option:", "Upload Choice", choices)

        if user_choice == "1. Image Upload":
            # Handle image upload
            file_path = image_file_explore()
            if not file_path:
                continue  # Return to the main menu

            # Load and process the image
            frame = cv2.imread(file_path)
            if frame is None:
                eg.msgbox("Failed to load the image file.", "Error")
                continue  # Return to the main menu

            # Perform Skin Mask Detection
            mask = detect_faces_with_skin_mask(frame)
            cv2.imshow('Skin Color Mask', mask)

            # Perform Skin Region Detection
            skin = detect_faces_with_skin_region(frame)
            cv2.imshow('Skin Color Region', skin)

            # Display dynamically calculated Skin-based ROI
            skin_roi, bound_box = capture_skin_based_roi(frame)
            if skin_roi is not None:
                cv2.imshow("Skin-based ROI", skin_roi)

                # Draw bounding box on the original image
                x, y, w, h = bound_box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imshow("Original Image with ROI", frame)

            # Wait for user input to close windows
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        elif user_choice == "2. Video Upload":
            # Handle video upload
            file_path = video_file_explore()
            if not file_path:
                continue  # Return to the main menu

            # Open the video file
            cap = cv2.VideoCapture(file_path)

            # Retrieve the frame rate of the video to set the playback speed
            fps = cap.get(cv2.CAP_PROP_FPS)
            delay = int(1000 / fps) if fps > 0 else 33  # Default to ~30 fps if FPS info is unavailable

            while True:  # Outer loop for continuous replay
                # Reset video to the beginning
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                while cap.isOpened():
                    # Read each frame from the video
                    ret, frame = cap.read()
                    if not ret:
                        print("Reached end of video.")
                        break


                    # Perform Skin Mask Detection
                    mask = detect_faces_with_skin_mask(frame)
                    cv2.imshow('Skin Color Mask', mask)

                    # Perform Skin Region Detection
                    skin = detect_faces_with_skin_region(frame)
                    cv2.imshow('Skin Color Region', skin)


                    # Display dynamically calculated Skin-based ROI
                    skin_roi, bound_box = capture_skin_based_roi(frame)
                    if skin_roi is not None:
                        cv2.imshow("Skin-based ROI", skin_roi)
                        # Draw bounding box on the original frame
                        x, y, w, h = bound_box
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.imshow("Original Image with ROI", frame)

                    # Exit the display loop on pressing 'q'
                    if cv2.waitKey(delay) & 0xFF in [ord('q'), ord('Q')]:
                        break  # Exit the video playback loop

                # Ask the user if they want to replay the video
                replay_choice = eg.ynbox("Do you want to replay the video?", "Replay", ["Yes", "No"])
                if not replay_choice:
                    cap.release()
                    cv2.destroyAllWindows()
                    break  # Exit the outer replay loop

        elif user_choice == "3. Take Photo":
            frame = capture_image()
            frame = preprocessing(frame)
            
            if frame is None:
                eg.msgbox("Failed to load the image file.", "Error")
                continue  # Return to the main menu

            # Perform Skin Mask Detection
            mask = detect_faces_with_skin_mask(frame)

            # Perform Skin Region Detection
            skin = detect_faces_with_skin_region(frame)

            # Display dynamically calculated Skin-based ROI
            skin_roi, bound_box = capture_skin_based_roi(frame)
            if skin_roi is not None:
                cv2.imshow("Skin-based ROI", skin_roi)

                # Draw bounding box on the original image
                x, y, w, h = bound_box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imshow("Original Image with ROI", frame)

            # Wait for user input to close windows
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        elif user_choice == "4. Show Colour":
            frame = capture_image()
            frame = preprocessing(frame)
            frame2 = skin_colour(frame)
            
            cv2.imshow("Skin Colour", frame2)

            # Wait for user input to close windows
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        elif user_choice == "5. Exit":
            # Exit the program
            print("Exiting the program.")
            break

        else:
            eg.msgbox("Please make a valid selection.", "Error")
            continue  # Continue prompting if no valid choice is selected

    # Close any remaining windows and release resources
    cv2.destroyAllWindows()

main()


Exiting the program.
